In [ ]:
# Based on docs:
# https://docs.lightly.ai/self-supervised-learning/tutorials/package/tutorial_simclr_clothing.html

# Also, see:
# https://github.com/giakoumoglou/classification/blob/main/notebooks/main_simclr.ipynb
# https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_digits.html

## Imports

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from pathlib import Path
import mlflow
import numpy as np

import os

import matplotlib.pyplot as plt

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import CSVLogger, MLFlowLogger
import torch
import torchvision

from sklearn.decomposition import PCA

from lightly.data import LightlyDataset
from lightly.transforms import SimCLRTransform, utils

from itertools import product

from utils import generate_embeddings, prepare_mnist_images
from callbacks import LossLoggingCallback, HiddenDimensionsCheckingCallback
from models import SimCLRModelVariableEmbeddingSize

## Setup


In [ ]:
# DEBUG
# device = "cpu"
device = "cuda" if torch.cuda.is_available() else "cpu"

accelerator = "gpu" if device == "cuda" else "cpu"

print(f"Using device: {device}")
print(f"Using accelerator: {accelerator}")

In [ ]:
# input_size = 32  # laptop
input_size = 128  # PC
# batch_size = 64  # laptop
batch_size = 256  # PC

num_workers = 8
seed = 1
max_epochs = 20
num_ftrs = 32

path_to_data = Path("datasets/MNIST")
path_to_train_data = path_to_data / "train"
path_to_test_data = path_to_data / "test"

experiment_name = "simclr_mnist_variable_embedding_size"

Let's set the seed for our experiments



In [ ]:
pl.seed_everything(seed)

In [ ]:
mlflow.set_experiment(experiment_name)

## Config

Here, we define the configuration for our experiment, to be logged in mlflow.

In [ ]:
# classes_list_of_lists = [[0, 4, 9], [0, 2, 4, 6, 9], [0, 1, 2, 4, 5, 7, 8]]
# latent_space_sizes_list = [8, 16, 32, 64, 128, 256, 512]

classes_list_of_lists = [[0, 1, 2, 4, 5, 7, 8]]
# latent_space_sizes_list = [32, 64, 128, 256, 512]
latent_space_sizes_list = [512]

projections_head_hidden_dims_list = [512]  # fixed for now

# initial_num_components_pca_list = [50, 20, 7, 3]   # not used here
initial_num_components_pca_list = [20]

In [ ]:
csv_logger = CSVLogger(save_dir="lightning_logs", name=experiment_name)
mlflow_logger = MLFlowLogger(experiment_name=experiment_name)

# Main loop

In [ ]:
for (
    classes_list,
    initial_num_components_pca,
    latent_space_size,
    projection_head_hidden_dim,
) in product(
    classes_list_of_lists,
    initial_num_components_pca_list,
    latent_space_sizes_list,
    projections_head_hidden_dims_list,
):
    with mlflow.start_run():

        # Log parameters
        mlflow.log_param("classes_list", classes_list)
        mlflow.log_param("initial_num_components_pca", initial_num_components_pca)
        mlflow.log_param("initial_latent_space_size", latent_space_size)
        mlflow.log_param("initial_projection_head_hidden_dim", projection_head_hidden_dim)
        mlflow.log_param("max_epochs", max_epochs)
        mlflow.log_param("batch_size", batch_size)

        # Prepare the data
        prepare_mnist_images(classes_list, path_to_data)

        ## Train
        train_transform = SimCLRTransform(
            input_size=input_size, vf_prob=0.5, rr_prob=0.5
        )
        train_dataset_simclr = LightlyDataset(
            input_dir=path_to_train_data, transform=train_transform
        )
        dataloader_train_simclr = torch.utils.data.DataLoader(
            train_dataset_simclr,
            batch_size=batch_size,
            shuffle=True,
            drop_last=True,
            num_workers=num_workers,
        )
        mlflow.log_metric("train_dataset_length", (len(train_dataset_simclr)))

        ## Test
        test_transform = torchvision.transforms.Compose(
            [
                torchvision.transforms.Resize((input_size, input_size)),
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize(
                    mean=utils.IMAGENET_NORMALIZE["mean"],
                    std=utils.IMAGENET_NORMALIZE["std"],
                ),
            ]
        )
        test_dataset_simclr = LightlyDataset(
            input_dir=path_to_test_data, transform=test_transform
        )
        dataloader_test_simclr = torch.utils.data.DataLoader(
            test_dataset_simclr,
            batch_size=batch_size,
            shuffle=False,
            drop_last=True,
            num_workers=num_workers,
        )
        mlflow.log_metric("test_dataset_length", (len(test_dataset_simclr)))

        # Create model and callbacks
        model = SimCLRModelVariableEmbeddingSize(
            latent_space_size=latent_space_size,
            projection_head_hidden_dim=projection_head_hidden_dim,
        )

        early_stopping_callback = EarlyStopping(
            monitor="train_loss_ssl", mode="min", patience=5
        )

        model_checkpoint_callback = ModelCheckpoint(
            monitor="train_loss_ssl", mode="min", save_top_k=-1
        )
        loss_logging_callback = LossLoggingCallback()
        hidden_dimensions_checking_callback = HiddenDimensionsCheckingCallback()

        # Train the model
        trainer = pl.Trainer(
            log_every_n_steps=10,
            max_epochs=max_epochs,
            devices=1,
            accelerator=accelerator,
            callbacks=[
                early_stopping_callback,
                model_checkpoint_callback,
                loss_logging_callback,
                hidden_dimensions_checking_callback,
            ],
            # logger=mlflow_logger,
            logger=csv_logger,
        )
        trainer.fit(model, dataloader_train_simclr)

        # Generate embeddings
        model.eval()
        embeddings, filenames = generate_embeddings(model, dataloader_test_simclr)

        # Visualize embeddings
        pca = PCA(n_components=initial_num_components_pca)
        data_pca = pca.fit_transform(embeddings.cpu().numpy())

        tsne = TSNE(n_components=2)
        data_tsne = tsne.fit_transform(data_pca)

        targets = [int(fname.split("\\")[0]) for fname in filenames]  # windows
        os.makedirs("outputs", exist_ok=True)

        # Log embeddings params
        mlflow.log_param(
            "actual_latent_space_size_sanity_check", hidden_dimensions_checking_callback.latent_space_size
        )
        mlflow.log_param(
            "actual_projection_head_hidden_dim_sanity_check",
            hidden_dimensions_checking_callback.projection_head_hidden_dim,
        )

        # Log metrics
        mlflow.log_metric(
            "final_train_loss_ssl", loss_logging_callback.train_epoch_losses[-1]
        )
        mlflow.log_metric(
            "best_train_loss_ssl", min(loss_logging_callback.train_epoch_losses)
        )

        # Save artifacts
        explained_variance_ratio = pca.explained_variance_ratio_
        explained_variance_path = f"outputs/explained_variance_{classes_list}_{initial_num_components_pca}_{latent_space_size}.png"
        plt.bar(
            range(1, len(explained_variance_ratio) + 1),
            explained_variance_ratio,
            alpha=0.4,
            align="center",
        )
        plt.step(
            range(1, len(explained_variance_ratio) + 1),
            np.cumsum(explained_variance_ratio),
            where="mid",
        )
        plt.xlabel("Principal Component Index")
        plt.ylabel("Explained Variance Ratio")
        plt.title("PCA Explained Variance Ratio")
        plt.savefig(explained_variance_path)
        plt.close()
        mlflow.log_artifact(explained_variance_path)

        tsne_path = f"outputs/tsne_{classes_list}_{initial_num_components_pca}_{latent_space_size}.png"
        plt.scatter(
            data_tsne[:, 0], data_tsne[:, 1], c=targets, cmap="viridis", alpha=0.4
        )
        plt.title("t-SNE visualization")
        plt.colorbar()
        plt.savefig(tsne_path)
        plt.close()
        mlflow.log_artifact(tsne_path)

        pca_path = f"outputs/pca_{classes_list}_{initial_num_components_pca}_{latent_space_size}.png"
        plt.scatter(
            data_pca[:, 0], data_pca[:, 1], c=targets, cmap="viridis", alpha=0.4
        )
        plt.title("PCA visualization")
        plt.colorbar()
        plt.savefig(pca_path)
        plt.close()
        mlflow.log_artifact(pca_path)

        loss_plot_path = f"outputs/loss_plot_{classes_list}_{initial_num_components_pca}_{latent_space_size}.png"
        plt.plot(
            range(len(loss_logging_callback.train_epoch_losses)),
            loss_logging_callback.train_epoch_losses,
            marker="o",
        )
        plt.xlabel("Epoch")
        plt.ylabel("Loss")
        plt.title("Training Loss (Epoch)")
        plt.savefig(loss_plot_path)
        plt.close()
        mlflow.log_artifact(loss_plot_path)